In [3]:
import nltk
import ast
import re
import copy
import math
import openpyxl
from collections import Counter
import pandas as pd
import numpy as np
import json
import tensorflow as tf 
import tensorflow_hub as hub
import csv
import random
import sklearn
import tensorflow_text
import gensim
from gensim.models import KeyedVectors



In [116]:
request, intent, attributes = [], [], []

# temp = pd.read_excel('/Users/sumanth.gurram/Desktop/en-itsm-zeroshot-attributes.xlsx')
# numRows, numCols = temp.shape
# numCols = numCols - 2
# for index, row in temp.iterrows():
#     request.append(row['English Utterance'])
#     intent.append(row['Intent'])
#     vec = row[2:]
#     vec = vec.tolist()
#     attributes.append(vec)

def input(file):
    data = json.load(open(file))['intents']
    for each in data:
        label = each['name']
        for req in each['samples']:
            r = req['utterance']
            if r == 'wi-fi':
                r = "wireless access"
            r = r.replace("-", " ")
            request.append(r)
            intent.append(label)
            attributes.append([0]*52)
            
            
input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/en-US-ITSM.json')
# input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/fr-FR-ITSM.json')
# input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/de-DE-ITSM.json')
# input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/en-US-HR.json')
# input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/fr-FR-HR.json')
# input('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/models/de-DE-HR.json')
# input('/Users/sumanth.gurram/Desktop/surf200-model.json')


u2l = {}
u2v = {}
file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/en-US-ITSM-test.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/fr-FR-ITSM-test.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/de-DE-ITSM.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/en-US-HR-test.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/fr-FR-HR-test.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/git/ml/nlu-tools/src/main/resources/f-measure/test-data/de-DE-HR.txt', 'r') 
# file = open('/Users/sumanth.gurram/Desktop/SURF-200-test.txt', 'r') 


Lines = file.readlines()
Lines = Lines[1:]
it, rq = [], []

for line in Lines: 
    split = line.strip().split('\t')
    s = split[2]
    tent = s.replace("sn_surf_200.", "")
    it.append(tent)
    r = split[1]
    if r == 'wi-fi':
        r = "wireless access"
    r = r.replace("-", " ")
    rq.append(r)

print(len(request))
print(len(rq))
print(len(Counter(intent)))
print(len(Counter(it)))

1397
3110
27
27


In [117]:
rq[85] = "incident ticket"
rq[1512] = "incident ticket"
rq[365] = "change request"
rq[385] = "change request"
rq[875] = "encryption"

In [49]:
# model = KeyedVectors.load_word2vec_format(
#     '/Users/sumanth.gurram/GoogleNews-vectors-negative300.bin', binary=True)

In [118]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [47]:
# def ang(x,y):
#     nx = np.linalg.norm(x)
#     ny = np.linalg.norm(y)
#     cos = np.dot(x, y)/(nx * ny)
#     if cos > 1:
#         cos = 1
#     elif cos < -1:
#         cos = -1
#     return 1 - np.arccos(cos)/np.pi

# vec = model["sultan"]
# vec2 = model["emperor"]
# print(ang(vec, vec2))

0.667988982014136


In [119]:
request_words = [[w for w in r.lower().split()] for r in request]
rq_words = [[w for w in r.lower().split()] for r in rq]
vectors = []
vc = []

for utt in request_words:
#     temp = []
#     for w in utt:
#         try:
#             temp.append(model[w])
#         except:
#             burner = 1
#     if len(temp) == 0:
#         print(utt)
    temp = embed(utt)
    vectors.append(np.average(temp, axis=0))

count = 0
for utt in rq_words:
#     temp = []
#     for w in utt:
#         try:
#             temp.append(model[w])
#         except:
#             burner = 1
#     if len(temp) == 0:
#         print(count)
#         print(utt)
    temp = embed(utt)
    vc.append(np.average(temp, axis=0))
    count += 1

In [124]:
np.array(vectors).shape
np.array(vc).shape

(3110, 512)

In [121]:
tents = Counter(intent)
tts = Counter(it)
combined = list(tents+tts)

labelmap = {}
for i in range(len(combined)):
    labelmap[combined[i]] = i+1
intentmap = []
itmap = []

for a in range(len(intent)):
    intentmap.append(labelmap[intent[a]])
    
for b in range(len(it)):
    itmap.append(labelmap[it[b]])
    

request = np.array(request)
intent = np.array(intent)
intentmap = np.array(intentmap)
vectors = np.array(vectors)

rq = np.array(rq)
it = np.array(it)
itmap = np.array(itmap)
vc = np.array(vc)

In [122]:
print(intentmap)
print(list(Counter(intentmap)))

[ 1  1  1 ... 27 27 27]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [125]:
uttlab = {}
uttvec = {}
uttatt = {}

for x in range(len(request)):
    r = request[x]
    i = intent[x]
    v = vectors[x]
    a = attributes[x]
    if r not in uttlab:
        uttlab[r] = i
    if r not in uttvec:
        uttvec[r] = np.array(v)
    if r not in uttatt:
        uttatt[r] = np.array(a)

for x in range(len(rq)):
    rr = rq[x]
    ii = it[x]
    vv = vc[x]
    if rr not in u2l:
        u2l[rr] = ii
    if rr not in u2v:
        u2v[rr] = np.array(vv)

In [126]:
import pickle
bw = "bow_guse_"
lg = "en_"
# lg = "fr_"
# lg = "de_"
# ty = "sf_"
ty = "it_"
# ty = "hr_"
c = bw + lg + ty
with open(c + 'train_uttlab.pickle', 'wb') as handle:
    pickle.dump(uttlab, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(c + 'train_uttvec.pickle', 'wb') as hd:
    pickle.dump(uttvec, hd, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(c + 'train_uttatt.pickle', 'wb') as hd:
    pickle.dump(uttatt, hd, protocol=pickle.HIGHEST_PROTOCOL)

with open(c + 'test_uttlab.pickle', 'wb') as handle:
    pickle.dump(u2l, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(c + 'test_uttvec.pickle', 'wb') as hd:
    pickle.dump(u2v, hd, protocol=pickle.HIGHEST_PROTOCOL)